# About: KVMのインストール-Report

KVMをインストールしたマシンに関して、事後条件を満たしているか(KVMが正しくインストールされ、予期した通りに動作しているか)を確認するためのNotebook。

## Operation Note


*ここに経緯を記述*

# Notebookと環境のBinding

Inventory中のgroup名でBind対象を指示する。
**KVMなどがインストールされた物理マシン**を示すInventory中の名前を以下に指定する。

In [1]:
target_group = 'test-hypervisor'

Bind対象への疎通状態を確認する。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


# 状態の検証


## 仮想マシン用ブリッジが作成されていること

仮想マシン用のブリッジが作成されていること。お手本を作成している環境においては、以下のようなインタフェース構成となることを想定している。

- ブリッジ br-eth1 インタフェース ... ここにはサービス用IPアドレスが設定される
- eth1インタフェース ... Promiscuousモードでサービス用NICと対応付け、br-eth1インタフェースに接続される

In [3]:
external_nic = 'eth1'
bridge_nic = 'br-eth1'

In [4]:
!ansible -a "/sbin/ip addr show {bridge_nic}" {target_group}
!ansible -a "/sbin/ip addr show {external_nic}" {target_group}
!ansible -a "/usr/sbin/brctl show {bridge_nic}" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
10: br-eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UNKNOWN 
    link/ether XX:XX:XX:XX:XX:XX brd XX:XX:XX:XX:XX:XX
    inet XXX.XXX.XXX.105/26 brd XXX.XXX.XXX.127 scope global br-eth1
    inet6 XX:XX:XX:XX:XX:XX/64 scope link 
       valid_lft forever preferred_lft forever

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
2: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP qlen 1000
    link/ether XX:XX:XX:XX:XX:XX brd XX:XX:XX:XX:XX:XX
    inet6 XX:XX:XX:XX:XX:XX/64 scope link 
       valid_lft forever preferred_lft forever

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
bridge name	bridge id		STP enabled	interfaces
br-eth1		8000.246e960db538	no		eth1
							vnet0



**br-bond0, bond0が定義されており、br-bond0にサービス用IPアドレスが定義されていれば**、仮想マシン操作に関するお手本Notebookはそのまま利用することができる。異なるNICの定義方法を採っている場合は、適宜NIC名などを書き換えながら実施する必要がある。

## libvirtのNetwork設定が無効化されていること

defaultのNetwork設定が無効化されているかどうかを確認する。

In [5]:
!ansible -b -a 'virsh net-list --all' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Name                 State      Autostart     Persistent
--------------------------------------------------
default              inactive   no            yes



**defaultのstateがinactiveになっていて、かつautostartがnoになっていれば**OK。

## dnsmasqが起動していること

dnsmasqが実行されているかを確認する。

In [6]:
!ansible -b -a 'service dnsmasq status' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
dnsmasq (pid  74097) is running...dnsdomainname: Unknown host



**dnsmasq (pid  XXXXX) is running と表示されれば**OK。

## libvirtが動作していること

libvirtが動作しており、仮想マシン一覧が取得できるかどうかを確認する。

In [7]:
!ansible -b -a 'virsh list' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
 Id    Name                           State
----------------------------------------------------
 5     testvm-001                     running



**エラーメッセージが表示されなければ**OK。